# Galaxy Classifier - Data exploration

The aim of this project is to build a classifier that can infer the type of a galaxy given some input data. THese input data will be two fold:
- We will first build typical machine learning classifiers, such as the KNN, SVM, MLP etc... which take as input some processed numeric data
- In a second part of the project, we will create a convolutional deep neural network that will take as input a raw image of a galaxy

In this first section we use the Galaxy Zoo processed data, and explore its components and properties.

The data originate from:

- tables used: https://data.galaxyzoo.org/
- images and explanation : https://zenodo.org/record/3565489#.YBBFLPtKiUl
- context of the Galaxy Zoo project: https://www.zooniverse.org/projects/zookeeper/galaxy-zoo/about/results
- Decision Tree for classification : https://data.galaxyzoo.org/gz_trees/gz_trees.html

## Galaxy Zoo1 Catalog Data

This dataset contains information on galaxy classifications made by participants of the Galaxy Zoo project.
The file can be found at the following URL: <https://galaxy-zoo-1.s3.amazonaws.com/GalaxyZoo1_DR_table2.csv.gz>

This table gives classifications of galaxies which have spectra included in SDSS Data Release 7.
The fraction of the vote in each of the six categories is given, along with debiased votes in elliptical and spiral categories and
flags identifying systems as classified as spiral, elliptical or uncertain.

### Column Descriptions

1. OBJID: Unique identifier of the galaxy in the SDSS catalog.
2. RA: Right Ascension (in degrees) of the galaxy.
3. DEC: Declination (in degrees) of the galaxy.
4. NVOTE: Number of votes obtained for this galaxy.
5. P_EL: Probability that the galaxy is elliptical.
6. P_CW: Probability that the galaxy is a clockwise spiraled galaxy.
7. P_ACW: Probability that the galaxy is a anticlockwise spiraled galaxy.
8. P_EDGE: Probability that the galaxy is a galaxy with a blurred edge.
9. P_DK: Probability that the galaxy is a galaxy with a dominant nucleus.
10. P_MG: Probability that the galaxy is a galaxy with multiple nuclei.
11. P_CS: Probability that the galaxy is a galaxy with strange features.
12. P_EL_DEBIASED: Probability of the elliptical classification, corrected for bias effect.
13. P_CS_DEBIASED: Probability of the classification with strange features, corrected for bias effect.
14. SPIRAL: Number of votes for the "spiral" classification.
15. ELLIPTICAL: Number of votes for the "elliptical" classification.
16. UNCERTAIN: Number of votes for the "uncertain" classification.

Each of these columns is used to describe the classification of a given galaxy by Galaxy Zoo participants.
The P_* columns contain classification probabilities for each type of galaxy, while the SPIRAL, ELLIPTICAL,
and UNCERTAIN columns provide the number of votes for each classification. The P_*_DEBIASED columns are corrected for bias effect.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('../data/GalaxyZoo1_DR_table2.csv')
df

,OBJID,RA,DEC,NVOTE,P_EL,P_CW,P_ACW,P_EDGE,P_DK,P_MG,P_CS,P_EL_DEBIASED,P_CS_DEBIASED,SPIRAL,ELLIPTICAL,UNCERTAIN
0,587727178986356823,00:00:00.41,-10:22:25.7,59,0.610,0.034,0.000,0.153,0.153,0.051,0.186,0.610,0.186,0,0,1
1,587727227300741210,00:00:00.74,-09:13:20.2,18,0.611,0.000,0.167,0.222,0.000,0.000,0.389,0.203,0.797,1,0,0
2,587727225153257596,00:00:01.03,-10:56:48.0,68,0.735,0.029,0.000,0.147,0.074,0.015,0.176,0.432,0.428,0,0,1
3,587730774962536596,00:00:01.38,+15:30:35.3,52,0.885,0.019,0.000,0.058,0.019,0.019,0.077,0.885,0.077,0,1,0
4,587731186203885750,00:00:01.55,-00:05:33.3,59,0.712,0.000,0.000,0.220,0.068,0.000,0.220,0.640,0.290,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667939,587727226763870322,23:59:58.76,-09:41:34.7,35,0.171,0.800,0.000,0.029,0.000,0.000,0.829,0.057,0.943,1,0,0
667940,587730775499407475,23:59:58.78,+15:49:01.3,21,0.810,0.048,0.000,0.095,0.048,0.000,0.143,0.758,0.193,0,0,1
667941,587727223024124280,23:59:58.81,+15:39:49.4,28,0.286,0.000,0.071,0.393,0.179,0.071,0.464,0.099,0.603,0,0,1
667942,587730774425600239,23:59:59.02,+15:09:18.8,23,0.391,0.000,0.043,0.000,0.130,0.435,0.043,0.390,0.045,0,0,1


**Selection of features**:  
The first column is a unique identifier which cannot be a feature for our model, and the second and third columns are the absolute positions of galaxies. We assume the universe to be homogenous and isotropic, so we will suppose that the position in the sky of the galaxy does not correlate with our classes/targets.

In [ ]:
data = df.drop(['OBJID','RA','DEC'], axis=1)

As this is a Galaxy classification model, we have to check the class imbalance in a dataset where we perform classification task, even though its class binary imbalance may have a major effect in the phase training, and ultimately on precision. To plot the value_counts for three-class columns, we can do it like the code below:

In [ ]:
plt.figure(figsize=(10,7))
plt.title('Count plot for Galaxy types ')
countplt = data[['SPIRAL','ELLIPTICAL','UNCERTAIN']]
sns.countplot(x="variable", hue='value', data=pd.melt(countplt))
plt.xlabel('Classes')
plt.show()

## Scaling the data

In [ ]:
from lblearn.datasets import load_galaxies

data = load_galaxies().data

NVOTE = data[:, 0]
low_percentil = 5
high_percentil = 95

q1 = np.percentile(NVOTE, low_percentil)
q2 = np.percentile(NVOTE, high_percentil)
print(f"{low_percentil}th percentile: {q1}, {high_percentil}th percentile: {q2}")

plt.figure()
plt.title("Distribution of 'NVOTE' values, with the 5th and 95th percentiles ")
plt.plot(NVOTE, '.', label=f'min={NVOTE.min()}, max={NVOTE.max()}, mean={NVOTE.mean()}')
plt.legend()
plt.axhline(y=q1, color='red', linestyle='--')
plt.axhline(y=q2, color='red', linestyle='--');

The `NVOTE` feature of our dataset has a very different scale and contains some outliers. These two characteristics lead to difficulties to visualize the data and, more importantly, they can degrade the predictive performance of many machine learning algorithms. Unscaled data can also slow down or even prevent the convergence of many gradient-based estimators.

Indeed many estimators are designed with the assumption that each feature takes values close to zero or more importantly that all features vary on comparable scales. In particular, metric-based and gradient-based estimators often assume approximately standardized data (centered features with unit variances). A notable exception are decision tree-based estimators that are robust to arbitrary scaling of the data.
(ref: https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html)

### Standard Scaler

Standard scaler removes the mean and scales the data to unit variance. The scaling shrinks the range of the feature values. However, the outliers have an influence when computing the empirical mean and standard deviation. Standard scaler therefore cannot guarantee balanced feature scales in the presence of outliers.

### Min Max Scaler
Min-Max Scaler rescales the data set such that all feature values are in the range [0, 1]. However, this scaling compresses all inliers.

Both standard scaler and min-max scaler are very sensitive to the presence of outliers.

### Max Abs Scaler

Max Abs Scaler is similar to Min-Max Scaler except that the values are mapped across several ranges depending on whether negative OR positive values are present. If only positive values are present, the range is [0, 1]. If only negative values are present, the range is [-1, 0]. If both negative and positive values are present, the range is [-1, 1]. On positive only data, both MinMaxScaler and MaxAbsScaler behave similarly. MaxAbsScaler therefore also suffers from the presence of large outliers.

### Robust Scaler

Unlike the previous scalers, the centering and scaling statistics of RobustScaler are based on percentiles and are therefore not influenced by a small number of very large marginal outliers. Consequently, the resulting range of the transformed feature values is larger than for the previous scalers. Note that the outliers themselves are still present in the transformed data.

### Power Transformer

Power Transformer applies a power transformation to each feature to make the data more Gaussian-like in order to stabilize variance and minimize skewness. Currently the Yeo-Johnson and Box-Cox transforms are supported and the optimal scaling factor is determined via maximum likelihood estimation in both methods. By default, PowerTransformer applies zero-mean, unit variance normalization. Note that Box-Cox can only be applied to strictly positive data.

### Quantile Transformer 

QuantileTransformer applies a non-linear transformation such that the probability density function of each feature will be mapped to a uniform or Gaussian distribution. In this case, all the data, including outliers, will be mapped to a uniform distribution with the range [0, 1], making outliers indistinguishable from inliers.

RobustScaler and QuantileTransformer are robust to outliers in the sense that adding or removing outliers in the training set will yield approximately the same transformation. But contrary to RobustScaler, QuantileTransformer will also automatically collapse any outlier by setting them to the a priori defined range boundaries (0 and 1). This can result in saturation artifacts for extreme values.

### Normalizer

The Normalizer rescales the vector for each sample to have unit norm, independently of the distribution of the samples.

Given that all of our samples except for `NVOTE` are probilities in the range [0, 1], setting all vectors to the same legnth seems to be the best way to normalize our data before training. The normalizer seems the best option in our case.

## Treated dataset

To resume, we have an unbalanced dataset with a sample having large values and outliers. Also, we chose to remove from the training the OBJID, as well as the position of the galaxy in the sky. 
We rescale our dataset so the vector for each sample have unit norm. I have created a helper function to get a balanced dataset, and to reduce the classes to one column, taking values from 0 to 2. The label 0 corresponds to a spiral galaxy and the label 1 an elliptical one.

In [ ]:
from lblearn.datasets import load_galaxies
import pandas as pd

In [ ]:
galaxies = load_galaxies(n_samples=100) # We can choose how many datapoints we want from each galaxy type

df = pd.DataFrame(galaxies.data, columns=galaxies.feature_names)
df['class'] = galaxies.target
df

# Galaxy Zoo 2: Images from Original Sample

Willett, Kyle W.; Lintott, Chris J.; Bamford, Steven P.; Masters, Karen L.; Simmons, Brooke D.; Casteels, Kevin R. V.; Edmonson, Edward M.; Fortson, Lucy F.; Kaviraj, Sugata; Keel, William C.; Melvin, Thomas; Nichol, Robert C.; Raddick, M. Jordan; Schawinski, Kevin; Simpson, Robert J.; Skibba, Ramin A.; Smith, Arfon M.; Thomas, Daniel

The Galaxy Zoo team regularly receives requests for subject images for various versions of Galaxy Zoo, in order to facilitate other investigations, e.g. machine learning projects. This repository is an updated attempt to provide those in a way that is useful to the wider community.

The images here are meant to be used with the data tables available at data.galaxyzoo.org. They are the "original" sample of subject images in Galaxy Zoo 2 (Willett et al. 2013, MNRAS, 435, 2835, DOI: 10.1093/mnras/stt1458) as identified in Table 1 of Willett et al. and also in Hart et al. (2016, MNRAS, 461, 3663, DOI: 10.1093/mnras/stw1588). The original GZ2 subjects also gave the option to view an inverted version of the subject image; these inverted images are not provided but are easily reproducible from the included subject images. 

If you use this dataset, please cite Willett et al. (2013) as the general data release and also cite the DOI for this dataset; if you use the updated debiased tables from Hart et al. (2016) please cite that as well.

There are 243,434 images in total. This is off by about 0.08% from the total count in the tables - it's not clear what the cause of the discrepancy is, but we don't think the missing images have any particular sampling bias, so this sample should be useful for research.

The images are available in a single zip file (images_gz2.zip).

The most recent and reliable source for morphology measurements is "GZ2 - Table 1 - Normal-depth sample with new debiasing method – CSV" (from Hart et al. 2016), which is available at data.galaxyzoo.org. <font color="red"> To cross-reference the images with Table 1, this sample includes another CSV table (gz2_filename_mapping.csv) which contains three columns and 355,990 rows </font>. The columns are:

    objid: the Data Release 7 (DR7) object ID for each galaxy. This should match the first column in Table 1.
    sample: string indicating the subsampling of the galaxy.  
    asset_id: an integer that corresponds to the filename of the image in the zipped file linked above.

As an example row:

587722981742084144,original,16

The galaxy is 587722981741363294, which is in Table 1 and was identified by GZ2 volunteers as a barred spiral galaxy with a mild bulge and two tightly-wound arms (morphology='Sc2t'). It is in the original GZ2 sample, and can be found in the zipped file as 16.jpg. 

The overlap between the set of images, the attached table, and Table 1 is not 100%; there are a few rows in the tables that don't have a corresponding image. Again, it's not clear what the exact reason is for this, but we suggest just dropping any missing rows/images from your analysis unless you have a need for analyzing specific subjects. If you do need a 100% complete sample, you can obtain the missing images directly from SDSS. 

Based on spot checks the mappings between asset ID and DR7 object ID appear correct, but we strongly suggest that you pick some random images and verify on your own that the image seems to match the label/classifications that are listed in Table 1. 

If you have any issues using this dataset, please contact the Galaxy Zoo team, in particular Brooke Simmons (b.simmons@lancaster.ac.uk). Should Dr Simmons be unavailable, try contacting Karen Masters or Chris Lintott.

- the GZ team, 5 Dec 2019

## Filename and class identification

On veut ici crée une table de passage entre le nom de fichier (colonne: `asset_id`) et la classe (colonens: `SPIRAL` , `ELLIPTICAL`,  `UNCERTAIN`). À noter qu'on veut réduire les trois colonnes de chaque classe en une unqiue colonne `target`qui prend les valeurs 0 pour une galaxie spirale, 1 pour elliptique et 2 pour incertain.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

table1 = pd.read_csv("../data/GalaxyZoo1_DR_table2.csv")
filename_tables = pd.read_csv('../data/gz2_filename_mapping.csv')

In [ ]:
id_to_filename.head(5)

In [ ]:
id_to_filename.ELLIPTICAL = id_to_filename.ELLIPTICAL.replace(1, 2)
id_to_filename.UNCERTAIN = id_to_filename.UNCERTAIN.replace(1, 3)
id_to_filename['classe'] = id_to_filename[['SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']].max(axis=1)
data = id_to_filename[['asset_id', 'classe']].values

In [ ]:
from collections import Counter
from lblearn.datasets import balance_data

data[:, 1] = data[:, 1] - 1 
target = data[:, 1]
count = Counter(target)
min_class, min_val = count.most_common()[-1]
n_samples = min_val
samples = balance_data(y=target, n_samples=34481)
data = data[samples, :]

In [ ]:
table_passe = pd.DataFrame({'asset_id': data[:, 0], 'classe': data[:, 1]})
table_passe.to_csv("../data/table_passage.csv")